In [1]:
import time
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import preprocessing
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

mp.cpu_count()

4

In [2]:
from fencoding_CPUs import (save_to_csv,
                               reduce_mem_usage,
                               EmptyElim,
                               FEncoding,
                               OutlDetect,
                               FImputation
                               )

In [3]:
from pathlib import Path
# home-credit-default-risk tables
if not Path("data/application_train.csv").is_file():
    ! wget https://www.dropbox.com/s/j9xwcj9ixki5t2l/home-credit-default-risk.zip?dl=0 -O data.zip
    ! unzip -q data.zip
    ! rm data.zip
# default-of-credit-card-clients-dataset
if not Path("data/default_ucr.csv").is_file():
    ! wget https://www.dropbox.com/s/lj0d7qez18ea7dx/UCI_Credit_Card.csv?dl=0 -O default_ucr.csv

--2020-11-18 15:44:00--  https://www.dropbox.com/s/j9xwcj9ixki5t2l/home-credit-default-risk.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/j9xwcj9ixki5t2l/home-credit-default-risk.zip [following]
--2020-11-18 15:44:01--  https://www.dropbox.com/s/raw/j9xwcj9ixki5t2l/home-credit-default-risk.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucef566cdc7e3bcd7c19adda7aab.dl.dropboxusercontent.com/cd/0/inline/BDZxxv0nM8TgImuhhHBsUm_kCPSjYOAeWgyojsoPe4VV-4rhmTKSLngWERrZOPE66TO_ylpiqeREECKdzL0rZp9xFu0o-LsgZBUX3L6kTE42qqSuuec_-gX5EIdrY7GKkjU/file# [following]
--2020-11-18 15:44:01--  https://ucef566cdc7e3bcd7c19adda7aab.dl.dropboxusercontent.com/cd/0/inline/BDZxxv0nM8TgImuhhHBsUm_kCPSjYOAeWgyojsoPe4VV-4rhmTKSLngWERrZOPE66TO_ylpiqeREECKdzL0rZp

 73950K .......... .......... .......... .......... .......... 10% 11,2M 73s"unzip" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.

 74000K .......... .......... .......... .......... .......... 10% 7,57M 73s
 74050K .......... .......... .......... .......... .......... 10% 21,6M 73s
 74100K .......... .......... .......... .......... .......... 10% 9,51M 73s
 74150K .......... .......... .......... .......... .......... 10% 13,1M 73s
 74200K .......... .......... .......... .......... .......... 10% 9,88M 73s
 74250K .......... .......... .......... .......... .......... 10% 6,20M 73s
 74300K .......... .......... .......... .......... .......... 10% 15,7M 73s
 74350K .......... .......... .......... .......... .......... 10% 8,13M 73s
 74400K .......... .......... .......... .......... .......... 10% 29,2M 73s
 74450K .......... .......... .......... .......... .......... 10% 3,71M 73s
 74500K .......... .......... .......... .......... ....

123950K .......... .......... .......... .......... .......... 17% 9,47M 67s
124000K .......... .......... .......... .......... .......... 17% 12,9M 67s
124050K .......... .......... .......... .......... .......... 17% 14,5M 67s
124100K .......... .......... .......... .......... .......... 17% 23,0M 67s
124150K .......... .......... .......... .......... .......... 17% 4,40M 67s
124200K .......... .......... .......... .......... .......... 17% 10,1M 67s
124250K .......... .......... .......... .......... .......... 17% 22,7M 67s
124300K .......... .......... .......... .......... .......... 17% 7,29M 67s
124350K .......... .......... .......... .......... .......... 17% 22,1M 67s
124400K .......... .......... .......... .......... .......... 17% 15,1M 67s
124450K .......... .......... .......... .......... .......... 17% 1,88M 67s
124500K .......... .......... .......... .......... .......... 17% 19,9M 67s
124550K .......... .......... .......... .......... .......... 17% 8,25M 67s

191900K .......... .......... .......... .......... .......... 27% 2,72M 61s
191950K .......... .......... .......... .......... .......... 27% 2,13M 61s
192000K .......... .......... .......... .......... .......... 27% 11,7M 61s
192050K .......... .......... .......... .......... .......... 27% 2,98M 61s
192100K .......... .......... .......... .......... .......... 27% 3,60M 61s
192150K .......... .......... .......... .......... .......... 27% 4,31M 61s
192200K .......... .......... .......... .......... .......... 27% 5,05M 61s
192250K .......... .......... .......... .......... .......... 27% 4,72M 61s
192300K .......... .......... .......... .......... .......... 27% 5,19M 61s
192350K .......... .......... .......... .......... .......... 27% 5,55M 61s
192400K .......... .......... .......... .......... .......... 27% 3,68M 61s
192450K .......... .......... .......... .......... .......... 27% 16,0M 61s
192500K .......... .......... .......... .......... .......... 27% 3,61M 61s

248900K .......... .......... .......... .......... .......... 35% 42,2M 57s
248950K .......... .......... .......... .......... .......... 35% 31,8M 57s
249000K .......... .......... .......... .......... .......... 35% 45,8M 57s
249050K .......... .......... .......... .......... .......... 35% 40,2M 57s
249100K .......... .......... .......... .......... .......... 35% 49,7M 57s
249150K .......... .......... .......... .......... .......... 35% 19,8M 57s
249200K .......... .......... .......... .......... .......... 35% 18,0M 57s
249250K .......... .......... .......... .......... .......... 35% 5,49M 57s
249300K .......... .......... .......... .......... .......... 35% 20,9M 57s
249350K .......... .......... .......... .......... .......... 35% 12,8M 57s
249400K .......... .......... .......... .......... .......... 35% 11,9M 57s
249450K .......... .......... .......... .......... .......... 35% 19,6M 57s
249500K .......... .......... .......... .......... .......... 35% 8,19M 57s

...

409150K .......... .......... .......... .......... .......... 58% 9,28M 36s
409200K .......... .......... .......... .......... .......... 58% 11,3M 36s
409250K .......... .......... .......... .......... .......... 58% 9,16M 36s
409300K .......... .......... .......... .......... .......... 58% 12,7M 36s
409350K .......... .......... .......... .......... .......... 58% 8,97M 36s
409400K .......... .......... .......... .......... .......... 58% 16,8M 36s
409450K .......... .......... .......... .......... .......... 58% 19,7M 36s
409500K .......... .......... .......... .......... .......... 58% 10,6M 36s
409550K .......... .......... .......... .......... .......... 58% 17,8M 36s
409600K .......... .......... .......... .......... .......... 58% 14,2M 36s
409650K .......... .......... .......... .......... .......... 58% 12,3M 36s
409700K .......... .......... .......... .......... .......... 58% 11,5M 36s
409750K .......... .......... .......... .......... .......... 58% 5,76M 36s

479750K .......... .......... .......... .......... .......... 68% 9,97M 27s
479800K .......... .......... .......... .......... .......... 68% 6,61M 27s
479850K .......... .......... .......... .......... .......... 68% 7,58M 27s
479900K .......... .......... .......... .......... .......... 68% 8,40M 27s
479950K .......... .......... .......... .......... .......... 68% 17,5M 27s
480000K .......... .......... .......... .......... .......... 68% 21,8M 27s
480050K .......... .......... .......... .......... .......... 68% 5,37M 27s
480100K .......... .......... .......... .......... .......... 68% 22,2M 27s
480150K .......... .......... .......... .......... .......... 68% 18,6M 27s
480200K .......... .......... .......... .......... .......... 68% 20,2M 27s
480250K .......... .......... .......... .......... .......... 68% 16,0M 27s
480300K .......... .......... .......... .......... .......... 68% 8,69M 27s
480350K .......... .......... .......... .......... .......... 68% 10,6M 27s

562600K .......... .......... .......... .......... .......... 79% 19,0M 17s
562650K .......... .......... .......... .......... .......... 79% 11,9M 17s
562700K .......... .......... .......... .......... .......... 79% 15,3M 17s
562750K .......... .......... .......... .......... .......... 79% 19,2M 17s
562800K .......... .......... .......... .......... .......... 79% 24,8M 17s
562850K .......... .......... .......... .......... .......... 79% 14,2M 17s
562900K .......... .......... .......... .......... .......... 79% 11,7M 17s
562950K .......... .......... .......... .......... .......... 79% 7,05M 17s
563000K .......... .......... .......... .......... .......... 79% 1,45M 17s
563050K .......... .......... .......... .......... .......... 79% 27,8M 17s
563100K .......... .......... .......... .......... .......... 79% 35,0M 17s
563150K .......... .......... .......... .......... .......... 79% 32,8M 17s
563200K .......... .......... .......... .......... .......... 79% 6,71M 17s

644200K .......... .......... .......... .......... .......... 91% 11,6M 7s
644250K .......... .......... .......... .......... .......... 91% 25,4M 7s
644300K .......... .......... .......... .......... .......... 91% 19,0M 7s
644350K .......... .......... .......... .......... .......... 91% 8,36M 7s
644400K .......... .......... .......... .......... .......... 91% 15,5M 7s
644450K .......... .......... .......... .......... .......... 91% 7,74M 7s
644500K .......... .......... .......... .......... .......... 91% 12,4M 7s
644550K .......... .......... .......... .......... .......... 91% 4,10M 7s
644600K .......... .......... .......... .......... .......... 91% 5,18M 7s
644650K .......... .......... .......... .......... .......... 91% 2,11M 7s
644700K .......... .......... .......... .......... .......... 91% 20,9M 7s
644750K .......... .......... .......... .......... .......... 91% 23,3M 7s
644800K .......... .......... .......... .......... .......... 91% 34,0M 7s
644850K ....

--2020-11-18 15:45:29--  https://www.dropbox.com/s/lj0d7qez18ea7dx/UCI_Credit_Card.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/lj0d7qez18ea7dx/UCI_Credit_Card.csv [following]
--2020-11-18 15:45:29--  https://www.dropbox.com/s/raw/lj0d7qez18ea7dx/UCI_Credit_Card.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbfc7cd23b95d67a99754730d11.dl.dropboxusercontent.com/cd/0/inline/BDZXQZezmXoKiwG28Lj6rFxNZlIVpH4L88-35Y_I3qmJMqzDAbt4IQPrsUMb05RT0VycAxri8eVhllyl4UOdHNsk9n-PTv_ZnUqXs-KPTiCON762m_ViDjDgb5KhyYZjpxI/file# [following]
--2020-11-18 15:45:30--  https://ucbfc7cd23b95d67a99754730d11.dl.dropboxusercontent.com/cd/0/inline/BDZXQZezmXoKiwG28Lj6rFxNZlIVpH4L88-35Y_I3qmJMqzDAbt4IQPrsUMb05RT0VycAxri8eVhllyl4UOdHNsk9n-PTv_ZnUqXs-KPTiCON762m

In [6]:
# Read in the source datasets
dict = {
    'datasets':[
                # home-credit-default-risk tables
                pd.read_csv('./data/application_train.csv'),
                # default-of-credit-card-clients-datasets
                pd.read_csv('./data/default_ucr.csv'),
    ],
    'name_dropped_columns':
                [
                 # home-credit-default-risk tables
                 ['SK_ID_CURR', 'TARGET'],
                 # default-of-credit-card-clients-datasets
                 ['ID', 'default.payment.next.month']              
    ],   
}
# Keep ID and target columns separately
dict['dropped_columns'] = [dict['datasets'][i][dict['name_dropped_columns'][i]] for i in range(len(dict['datasets']))]
# Drop ID and target columns from the tables
dict['datasets'] = [dict['datasets'][i].drop(dict['name_dropped_columns'][i], axis=1) for i in range(len(dict['datasets']))]

In [7]:
# default-of-credit-card-clients-datasets
X, y = dict['datasets'][1], dict['dropped_columns'][1]['default.payment.next.month']
X = reduce_mem_usage(X)
#fencoding.initialize_types(X)


Memory usage of dataframe is 5.26 MB
Memory usage after optimization is: 1.77 MB
Memory usage decreased by 66.3%


In [8]:
emptyelim = EmptyElim(n_jobs= -1, chunks = None)
fencoding = FEncoding(n_jobs = -1, chunks = None)
outldetect = OutlDetect(outliers_detection_technique = 'iqr_proximity_rule', n_jobs = -1, 
                                                                             chunks = None)

# For regression
fimputation_rb = FImputation('regression-based', fill_with_value = None, n_jobs = -1, chunks = None)
# For boosting
fimputation_tb = FImputation('tree-based', fill_with_value = 'extreme_values', n_jobs = -1, chunks = None)

In [9]:
fencoding = FEncoding(n_jobs = -1, chunks = None)

In [10]:
fencoding.initialize_types(X, return_dtype=True)

{'categor_columns': ['LIMIT_BAL',
  'SEX',
  'EDUCATION',
  'MARRIAGE',
  'AGE',
  'PAY_0',
  'PAY_2',
  'PAY_3',
  'PAY_4',
  'PAY_5',
  'PAY_6',
  'BILL_AMT1',
  'BILL_AMT2',
  'BILL_AMT3',
  'BILL_AMT4',
  'BILL_AMT5',
  'BILL_AMT6',
  'PAY_AMT1',
  'PAY_AMT2',
  'PAY_AMT3',
  'PAY_AMT4',
  'PAY_AMT5',
  'PAY_AMT6'],
 'numer_columns': [],
 'time_columns': [],
 'categor_columns_dtypes': ['float32',
  'int8',
  'int8',
  'int8',
  'int8',
  'int8',
  'int8',
  'int8',
  'int8',
  'int8',
  'int8',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32'],
 'numer_columns_dtypes': [],
 'time_columns_dtypes': []}

In [11]:
X = emptyelim.fit_transform(X)
X = fencoding.date_replace(X)
X = fencoding.encode_categor(X, method = 'OrdinalEncoder')
X = outldetect.fit_transform(X)


 columns to drop: {}

 No time columns in the dataset

 No numerical columns in the dataset

 No numerical columns in the dataset


In [12]:
XX = fimputation_rb.impute(X)
XX = reduce_mem_usage(XX)
save_to_csv(XX, y, path='data/default_ucr_regression.csv')

XX = fimputation_tb.impute(X)
XX = reduce_mem_usage(XX)
save_to_csv(XX, y, path='data/default_ucr_boosting.csv')


Memory usage of dataframe is 2.63 MB
Memory usage after optimization is: 1.00 MB
Memory usage decreased by 62.0%

 Successfully saved to data/default_ucr_regression.csv

Memory usage of dataframe is 2.63 MB
Memory usage after optimization is: 1.00 MB
Memory usage decreased by 62.0%

 Successfully saved to data/default_ucr_boosting.csv


In [13]:
# home-credit-default-risk tables
X, y = dict['datasets'][0], dict['dropped_columns'][0]['TARGET']
X = reduce_mem_usage(X)
#fencoding.initialize_types(X)


Memory usage of dataframe is 281.53 MB
Memory usage after optimization is: 90.91 MB
Memory usage decreased by 67.7%


In [14]:
emptyelim = EmptyElim(n_jobs= -1, chunks = None)
fencoding = FEncoding(n_jobs = -1, chunks = None)
outldetect = OutlDetect(outliers_detection_technique = 'iqr_proximity_rule', n_jobs = -1, 
                                                                             chunks = None)

# For regression
fimputation_rb = FImputation('regression-based', fill_with_value = None, n_jobs = -1, chunks = None)
# For boosting
fimputation_tb = FImputation('tree-based', fill_with_value = 'extreme_values', n_jobs = -1, chunks = None)

In [15]:
X = emptyelim.fit_transform(X)
X = fencoding.date_replace(X)
X = fencoding.encode_categor(X, method = 'OrdinalEncoder')
X = outldetect.fit_transform(X)


 columns to drop: {}

 No time columns in the dataset

 col_outl_info (upper, lower) bounds: {'AMT_INCOME_TOTAL': (-22500.0, 337500.0), 'AMT_CREDIT': (-537975.0, 1616625.0), 'AMT_ANNUITY': (-10584.0, 61704.0), 'AMT_GOODS_PRICE': (-423000.0, 1341000.0)}


In [16]:
XX = fimputation_rb.impute(X)
XX = reduce_mem_usage(XX)
save_to_csv(XX, y, path='data/application_train_regression.csv')

XX = fimputation_tb.impute(X)
XX = reduce_mem_usage(XX)
save_to_csv(XX, y, path='data/application_train_boosting.csv')

MemoryError: 